In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


In [2]:
import os
import geopandas as gpd
import pandas as pd

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel
)
from utils.downloads import _prepare_cache

# Intersecção entre setores censitários (2022) e a área de 1km a partir de Parques Urbanos e Lineares (buff)

Pessoas de outras cidades também estão dentro das áreas de 1km de parques, porém optei por não considerá-las ao fazer a contagem.

Eu interpretei o indicador como <=1km, não como <1km

In [3]:
data_path = get_data_diretorio()

In [4]:
assets_path = os.path.join(
    data_path,
    'assets'
)

# Dependências

Este notebook depende dos arquivos resultantes dos notebooks '.buffed_parques_urb_line' e '../load_data/setores_censitario'

In [5]:
gdf_buffed_parq = gpd.read_parquet(
    os.path.join(
        assets_path,
        'parques_lineares_e_urbanos',
        'area_1km_parques_urbanos_lineares.parquet'        
    )
)

In [6]:
gdf_setores22= gpd.read_parquet(
    os.path.join(
        assets_path,
        'setores_2022.parquet'
    )
)

In [7]:
gdf_setores22.shape

(27301, 27)

# Overlay setores por buff

In [8]:
gdf_setores22['area_setor']=gdf_setores22['geometry'].area

In [9]:
overlay_buff_setores= gpd.overlay(
    gdf_setores22,
    gdf_buffed_parq,
    how='intersection',
    keep_geom_type=True    
)

In [10]:
overlay_buff_setores.shape

(17752, 28)

# Calcular a população

Como não temos como saber precisamente onde as pessoas se encontram dentro do setore censitário, estabelecemos por padrão que faremos o cálculo como se elas estivessem distribuídas de forma homogênea no espaço. A partir disso, podemos assumir que a maior % de área do setor, também equivalerá a uma maior quantidade de pessoas neste recorte.

In [11]:
overlay_buff_setores['area_recort']=overlay_buff_setores['geometry'].area

Para apagarmos as geometrias desprezíveis eu usarei o mesmo padrão que usei para quadras viárias e outras malhas das Áreas Verdes Públicas: desprezar as geometrias menores que 1% da área total do menor setor

In [12]:
gdf_setores22.loc[gdf_setores22['area_setor']==gdf_setores22['area_setor'].min()]

,CD_SETOR,SITUACAO,CD_SIT,CD_TIPO,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,...,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,total_pop,total_dom,geometry,area_setor
22732,355030878000084,Urbana,1,0,3550308,São Paulo,355030878,Sé,35503087800,None,...,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,283.0,130.0,"POLYGON ((333744.46 7394168.456, 333741.319 73...",246.391634


In [13]:
min_setor= float(gdf_setores22['area_setor'].min())
min_1pct = min_setor*0.01
irrelevants = overlay_buff_setores.loc[overlay_buff_setores['area_recort']<min_1pct]

In [14]:
irrelevants.shape

(16, 29)

In [15]:
overlay_buff_setores=overlay_buff_setores.loc[overlay_buff_setores['area_recort']>=min_1pct]

In [16]:
overlay_buff_setores.shape

(17736, 29)

In [17]:
overlay_buff_setores['percent_area'] = overlay_buff_setores['area_recort']/overlay_buff_setores['area_setor']

Agora que sabemos a porcentagem da área de cada recorte, podemos multiplicá-la (já que está em decimal) pelo total da população no setor e teremos a quantidade de pessoas nesta "fatia". Repare que, por estarmos calculando o número de pessoas, é necessário que seja um número inteiro (no caso da linguagem, um número arredondado a 0 casa decimais) 

In [18]:
overlay_buff_setores['pop_recort']=round(overlay_buff_setores['total_pop']*overlay_buff_setores['percent_area'], 0)

In [19]:
overlay_buff_setores.sample(2)

,CD_SETOR,SITUACAO,CD_SIT,CD_TIPO,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,...,NM_RGI,CD_CONCURB,NM_CONCURB,total_pop,total_dom,area_setor,geometry,area_recort,percent_area,pop_recort
6986,355030837000403,Urbana,1,0,3550308,São Paulo,355030837,Itaquera,35503083700,None,...,São Paulo,3550308,São Paulo/SP,706.0,275.0,36316.508145,"MULTIPOLYGON (((350251.542 7398014.194, 350265...",1805.240777,0.049709,35.0
11715,355030865000088,Urbana,1,0,3550308,São Paulo,355030865,Raposo Tavares,35503086500,None,...,São Paulo,3550308,São Paulo/SP,299.0,157.0,26942.207644,"POLYGON ((317175.226 7388909.231, 317048.076 7...",26942.207644,1.000000,299.0


Para visualizar melhor, vamos procurar só os casos que o setor não está completo:

In [20]:
overlay_buff_setores.loc[overlay_buff_setores['percent_area']<=0.99].sample(4)

,CD_SETOR,SITUACAO,CD_SIT,CD_TIPO,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,...,NM_RGI,CD_CONCURB,NM_CONCURB,total_pop,total_dom,area_setor,geometry,area_recort,percent_area,pop_recort
15882,355030887000113,Urbana,1,0,3550308,São Paulo,355030887,Vila Jacuí,35503088700,None,...,São Paulo,3550308,São Paulo/SP,489.0,197.0,36890.266207,"POLYGON ((351327.445 7398817.914, 351338.005 7...",13904.768342,0.376922,184.0
17469,355030895000272,Urbana,1,1,3550308,São Paulo,355030895,São Domingos,35503089500,None,...,São Paulo,3550308,São Paulo/SP,166.0,80.0,49534.377029,"POLYGON ((319667.078 7402444.853, 319687.183 7...",23868.615662,0.481860,80.0
13873,355030876000099,Urbana,1,0,3550308,São Paulo,355030876,Sapopemba,35503087600,None,...,São Paulo,3550308,São Paulo/SP,672.0,243.0,20656.190126,"POLYGON ((346756.092 7389540.389, 346729.061 7...",20390.658817,0.987145,663.0
1152,355030811000279,Urbana,1,0,3550308,São Paulo,355030811,Brasilândia,35503081100,None,...,São Paulo,3550308,São Paulo/SP,361.0,168.0,28350.015945,"POLYGON ((327534.915 7403506.884, 327524.731 7...",9583.419237,0.338039,122.0


# Salvar gdf

In [21]:
save_parquet_excel(
    overlay_buff_setores,
    fname='overlay_setores22_buff_parques_urb_line',
    data_path = assets_path,
    data_subpath = 'parques_lineares_e_urbanos'
)